In [ ]:
import json
import os
import cv2
import numpy as np
import io
from PIL import Image
import segno

In [ ]:
config = json.load(open('/home/lei/rajrup/qr/KinectStream/Multiview/config/panoptic.json'))

seqName = config['seq_name']
inputdir = os.path.join(config['panoptic_path'], seqName)
outputdir = os.path.join(config['panoptic_path'], "qr")

if not os.path.exists(outputdir):
    os.makedirs(outputdir)

# temp = json.load(open(os.path.join(config['panoptic_path'], config['seq_start_end'])))
# startFrame = temp[seqName]['start_idx']
# endFrame = temp[seqName]['end_idx']

startFrame = 0
endFrame = 20000

In [ ]:
# QR code params
qrVersion = 4
qrErrorCorrection = 'H' # L, M, H (Low, Medium, High)
qrBorder = 4
qrScale = 2

# Only for view 0 for now
for frameID in range(startFrame, endFrame+1):
    print("Generating QR code for frame {}".format(frameID))
    
    # Generate QR code
    qrcode = segno.make(frameID, micro=False, version=qrVersion, error=qrErrorCorrection)
    # qrcode.save(os.path.join(outputdir, "{}_qrcode.png".format(frameID)), border=4, scale=2, dark='#000000', light='#FFFFFF')
    
    # In-memory PIL object
    out = io.BytesIO()
    qrcode.save(out, border=qrBorder, scale=qrScale, dark='#000000', light='#FFFFFF', kind='png')
    out.seek(0)
    pil_img = Image.open(out)
    # pil_img.save(os.path.join(outputdir, "{}_qrcode.png".format(frameID)), format='png')
    # pil_img.save(os.path.join(outputdir, "{}_qrcode.jpg".format(frameID)), format='jpeg', quality=1)

    cv2_img = np.array(pil_img) * 255.0
    cv2.imwrite(os.path.join(outputdir, "{}_qrcode.png".format(frameID)), cv2_img)
    # cv2.imwrite(os.path.join(outputdir, "{}_qrcode.jpg".format(frameID)), cv2_img, [int(cv2.IMWRITE_JPEG_QUALITY), 0])
    
    h, w = cv2_img.shape
    borderOffset = qrBorder*qrScale
    cv2_crop_img = cv2_img[borderOffset:h-borderOffset, borderOffset:w-borderOffset]
    cv2.imwrite(os.path.join(outputdir, "{}_qrcode_cropped.png".format(frameID)), cv2_crop_img)
    # cv2.imwrite(os.path.join(outputdir, "{}_qrcode_cropped.jpg".format(frameID)), cv2_crop_img, [int(cv2.IMWRITE_JPEG_QUALITY), 0])